# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import numpy as np
import pandas as pd
# import requests
# import joblib
# import json
# import logging

# from sklearn.metrics import confusion_matrix
# from sklearn.utils import resample
# from sklearn.model_selection import train_test_split


# from azureml.core.environment import Environment
# from azureml.core.conda_dependencies import CondaDependencies
# from azureml.core.experiment import Experiment
# from azureml.core.workspace import Workspace
# from azureml.core.dataset import Dataset
# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.compute import AmlCompute
# from azureml.core.compute import ComputeTarget
# from azureml.core.compute_target import ComputeTargetException
# from azureml.core.model import Model
# from azureml.widgets import RunDetails
# from azureml.train.automl import AutoMLConfig
# from azureml.core.webservice import LocalWebservice
# from azureml.core.webservice import Webservice, AciWebservice
# from azureml.core.model import InferenceConfig
# from azureml.automl.core.shared import constants


## Dataset

### Overview

The credit approval data is taken from UCI data repository. We will use the data to train the model and predict to approve credit or not.

All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. 

This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:


A1: b, a. \
A2: continuous. \
A3: continuous. \
A4: u, y, l, t. \
A5: g, p, gg. \
A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff. \
A7: v, h, bb, j, n, z, dd, ff, o. \
A8: continuous. \
A9: t, f. \
A10: t, f. \
A11: continuous. \
A12: t, f. \
A13: g, p, s. \
A14: continuous. \
A15: continuous. \
A16: +,- (class attribute)

Data has 296 approved and 357 rejected applications.

This dataset is public and available for research.
Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.


In [60]:
def get_data():
    columns = ['A'+str(i) for i in range(1,17)]
    df = pd.read_csv('data.csv', header = None, names = columns )
    df.dropna(inplace = True)

    # Removing Null
    for column in df.columns:
        df = df[df[column] != '?']

    # Converting attributes to binary
    df['A16'] = df['A16'].map({'+': 1, '-':0})    
    df['A1']  = df['A1'].map({'b': 1, 'a':0})
    df['A9']  = df['A9'].map({'t': 1, 'f':0})
    df['A10'] = df['A10'].map({'t': 1, 'f':0})
    df['A12'] = df['A12'].map({'t': 0, 'f':1})

    # Conversting categorical data into onehot encoding
    cat_columns = ['A4', 'A5', 'A6', 'A7', 'A13']
    for column in cat_columns:
        dummies = pd.get_dummies(df[column])
        df[dummies.columns] = dummies
        df.drop(columns = column,inplace = True)
    df = df.astype(float)
    
    return df

In [64]:
df['A16'].value_counts()

0.0    357
1.0    296
Name: A16, dtype: int64

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)

In [61]:
dataset_name = 'credit_approval_data'

# Check Registry
if dataset_name in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_name] 

    print(f'Located {dataset_name} in Registry.')

# Read from UCI ML Repository
else:
    df = get_data()
    
    # Register the dataset
    datastore = ws.get_default_datastore()

    dataset = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df, 
#         name=dataset_name, 
        description='A dataset of credit approved and rejected applications',
        target=datastore
    )

    print(f'Read {dataset_name} from UCI ML Repository and registered dataset.')
    

(653, 39)

In [ ]:
# Spliting the DataFrame into training and testing datasets
train, test = train_test_split(df, random_state=99, shuffle=True, test_size=0.2)
datastore = ws.get_default_datastore()
train = Dataset.Tabular.register_pandas_dataframe(
        dataframe=train, 
        name='df_train', 
        description='training dataset',
        target=datastore
    )

## AutoML Configuration

*The experiment_timeout_minutes (20 mins), iterations (50) and enable_early_stopping to reduce time taken for model training. 
*Enabling early stopping allows the training process to conclude if there is no considerable improvement to the primary_metric.
*cross_validations:5 is set to prevent overfitting.
*The max_concurrent_iterations: 4 is used to allow jobs to be run in parallel on each node.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":20,
    "enable_early_stopping":True,    
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "iterations":50,
    "max_concurrent_iterations":4,
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task='classification', 
    training_data=train,
    label_column_name='A16',
    **automl_settings)


In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
status = remote_run.wait_for_completion()


In [ ]:
best_run, fitted_model = remote_run.get_output()


# Best Run
print("Best Run: ")
print("")

# Accuracy
print("Accuracy: " + str(best_run.get_metrics()['accuracy']))
print("")

# Metrics
print("Metrics: " + str(best_run.get_metrics()))
print("")

# Name
print("Name: " + str(best_run.properties['model_name']))
print("")

# Tags
print("Tags: " + str(best_run.get_tags()))
print("")

In [ ]:
#Save the best model
#TODO: Save the best model
joblib.dump(fitted_model, filename='automl_best_model.joblib')

# Register the best model
model_name = best_run.properties['model_name']
model = remote_run.register_model(model_name=model_name)


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_run.get_file_names()

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
